## imports



In [30]:
from os.path import join
from json import loads
from collections import defaultdict

## environment variables

In [3]:
DATABASE_NAME = join('.', "farmers-protest-tweets-2021-03-5.json")

## classes

In [34]:
class RefInt:
    def __init__(self, reference):
        self.ref = reference
    def __lt__(self, other):
        return self.ref < other.ref

# FEATURE CODE

 ## feature 1 code

In [36]:
def top10_retweets():
    top10 = list()
    ints = dict()
    def hook(dict_):
        if type(dict_) != dict:
            return dict_
        if "url" in dict_ and "retweetCount" in dict_:
            url = dict_["url"]
            retwcount = dict_["retweetCount"]
            if url not in ints:
                ints[url] = RefInt(retwcount)
                top10.append((ints[url], url))
            elif ints[url].ref < retwcount:
                ints[url].ref = retwcount
                if (ints[url], url) not in top10:
                    top10.append((ints[url], url))
            else:
                return {}
            top10.sort(reverse=True)
            del top10[10:]
            return {}
    with open(DATABASE_NAME, "r", encoding="utf-8") as file:
        for line in file:
            loads(f'{line.strip()}',  object_hook=hook)
    for n, url in top10:
        print(f"{n.ref:8d}, {url}")

  904547, https://twitter.com/MrBeast/status/1362997215308906498
  315557, https://twitter.com/rihanna/status/1356625889602199552
  236683, https://twitter.com/pulte/status/1365015962433564675
  152380, https://twitter.com/rihanna/status/254929287726116864
  137786, https://twitter.com/Twitter/status/1357000724593442816
  123338, https://twitter.com/KingSalman/status/1330103721506840578
  116239, https://twitter.com/elonmusk/status/1358542364948668418
  115492, https://twitter.com/elonmusk/status/1357244295091208193
  111737, https://twitter.com/rihanna/status/1351997615328661506
  103960, https://twitter.com/GretaThunberg/status/1356694884615340037


## feature 2 code

In [ ]:
def top10_twit_senders():
    pass

## feature 3 code

In [ ]:
def top10_twit_days():
    pass

## feature 4 code

In [ ]:
def top10_hashtags():
    pass

## main function code

In [ ]:
def Main():
    pass

# Main function execution

In [ ]:
Main()